In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
torchrec_root = '../'

import sys
sys.path.append(torchrec_root)

In [3]:
import random
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim import Adam

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Load data

In [6]:
import joblib

df_samples = joblib.load(f'{torchrec_root}/data/amazon_data.pkl')

In [7]:
len( df_samples['reviewerID'].unique() )

192403

In [8]:
len( df_samples['asin'].unique() )

63001

In [9]:
len( df_samples['brand'].unique() )

3526

In [10]:
len( df_samples['categories'].unique() )

801

In [11]:
# sequence features from latest to oldest
df_samples['his_asin_seq'] = df_samples['his_asin_seq'].map(lambda x: x[::-1])

## Create pytorch dataset

In [12]:
## Hash buckets
# feat_configs = [
#     {"name": "reviewerID", "dtype": "category", "emb_dim": 12, "min_freq": 3, "hash_buckets": 1000000},
#     {"name": "asin", "dtype": "category", "emb_dim": 12, "min_freq": 3, "hash_buckets": 1000000},
    
#     {"name": "price", "dtype": "numerical", "norm": "std"},
#     {"name": "brand", "dtype": "category", "min_freq": 3, "emb_dim": 12},
#     {"name": "categories", "dtype": "category", "min_freq": 3, "emb_dim": 12},

#     {"name": "his_asin_seq", "dtype": "category", "islist": True, "min_freq": 3, "emb_dim": 12, "hash_buckets": 1000000},
# ]

## Dynamic Embedding
# feat_configs = [
#     {"name": "reviewerID", "dtype": "category", "emb_dim": 12, "min_freq": 3},
#     {"name": "asin", "dtype": "category", "emb_dim": 12, "min_freq": 3},
    
#     {"name": "price", "dtype": "numerical", "norm": "std"},
#     {"name": "brand", "dtype": "category", "min_freq": 3, "emb_dim": 12},
#     {"name": "categories", "dtype": "category", "min_freq": 3, "emb_dim": 12},

#     {"name": "his_asin_seq", "dtype": "category", "islist": True, "min_freq": 3, "emb_dim": 12},
# ]

## Auto generate feat_configs
from torchrec.utils import auto_generate_feature_configs
feat_configs = auto_generate_feature_configs(
    df_samples[['reviewerID', 'asin', 'price', 'brand', 'categories', 'his_asin_seq']]
)

print(feat_configs)

target_cols = ['label', ]

[{'name': 'reviewerID', 'dtype': 'category', 'emb_dim': 17, 'min_freq': 3}, {'name': 'asin', 'dtype': 'category', 'emb_dim': 15, 'min_freq': 3}, {'name': 'price', 'dtype': 'numerical', 'norm': 'std', 'mean': np.float64(74.40153304932919), 'std': np.float64(123.75264929565961)}, {'name': 'brand', 'dtype': 'category', 'emb_dim': 11, 'min_freq': 3}, {'name': 'categories', 'dtype': 'category', 'emb_dim': 9, 'min_freq': 3}, {'name': 'his_asin_seq', 'dtype': 'category', 'islist': True, 'emb_dim': 15, 'min_freq': 3, 'max_len': 256}]


In [13]:
from torchrec.sample import traintest_split

df_train, df_test = traintest_split(df_samples, test_size=0.2, shuffle=True, group_id='reviewerID')
print(len(df_train), len(df_test))

1352538 336650


In [14]:
# from torchrec.dataset import FeatureTransformer

# transformer = FeatureTransformer(feat_configs)

# df_train = transformer.transform(df_train, is_train=True, n_jobs=4)
# df_test = transformer.transform(df_test, is_train=False, n_jobs=4)

In [15]:
import polars as pl

df_train = pl.from_pandas(df_train)
df_test = pl.from_pandas(df_test)

In [16]:
from torchrec.dataset import DataFrameDataset

train_dataset = DataFrameDataset(df_train, feat_configs, target_cols, is_raw=True, is_train=True, n_jobs=1, verbose=True)

feat_configs = train_dataset.transformer.get_feat_configs()
test_dataset = DataFrameDataset(df_test, feat_configs, target_cols, is_raw=True, is_train=False, n_jobs=3)

==> Feature transforming (is_train=True), note that feat_configs will be updated when is_train=True...
Input dataFrame type: <class 'polars.dataframe.frame.DataFrame'>, transform it by FeatureTransformerPolars
Processing feature reviewerID...
Converting category reviewerID to indices...
Feature reviewerID vocab size: None -> 153923
Processing feature asin...
Converting category asin to indices...
Feature asin vocab size: None -> 62384
Processing feature price...
Feature price mean: 74.40153304932919, std: 123.75264929565961, min: 0.01, max: 999.99
Processing feature brand...
Converting category brand to indices...
Feature brand vocab size: None -> 3503
Processing feature categories...
Converting category categories to indices...
Feature categories vocab size: None -> 800
Processing feature his_asin_seq...
Converting category his_asin_seq to indices...
Feature his_asin_seq vocab size: None -> 61925
==> Feature transforming (is_train=True) done...
==> Dense features: ['price']
==> Sparse

In [17]:
# max([v['idx'] for k,v in feat_configs[3]['vocab'].items()])
# feat_configs[3]['num_embeddings']

In [18]:
df_train.head()

reviewerID,asin,unixReviewTime,overall,title,price,brand,categories,label,his_asin_seq
str,str,i64,f64,str,f64,str,str,i64,list[str]
"""A1PC34OXBBHDEF""","""B0000BZL5A""",1375660800,4.0,"""B+W 58mm Kaesemann Circular Po…",89.0,"""B+W""","""Polarizing Filters""",1,"[""B0000BZL1P""]"
"""A27A0U9O1HUSC8""","""B007IV7KRU""",1342137600,5.0,"""OtterBox Defender Series Case …",64.97,"""OtterBox""","""Cases""",1,"[""B004JMZZE6"", ""B005O22Y7G"", ""B004RDWVUS""]"
"""A1FMND912KUYSX""","""B006ZW4HY2""",1397088000,5.0,"""Olympus VN-702PC Voice Recorde…",58.98,"""Olympus""","""Digital Voice Recorders""",1,"[""B003ANVQWU"", ""B000RT77I2"", ""B000652M6Y""]"
"""A27LEATCCMJJF4""","""B001D2LJ3Q""",1357516800,4.0,"""Manfrotto 701HDV Pro Fluid Vid…",299.99,null,"""Tripod Heads""",1,"[""B005FYNSPK"", ""B003D5MZUW""]"
"""A1SMJ3J49A59FX""","""B008DWYBZM""",1365724800,4.0,"""Bear Motion Luxury Buffalo Hid…",49.99,"""Bear Motion""","""Cases""",1,"[""B005CLPP84"", ""B005U0M9B8"", ""B002L6HDTC""]"


In [19]:
train_dataloader = DataLoader(train_dataset, batch_size=512, num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=512, num_workers=2, shuffle=False, collate_fn=test_dataset.collate_fn)

In [20]:
print( len(train_dataloader) )
for features, labels in DataLoader(train_dataset, batch_size=2, num_workers=0, shuffle=True, collate_fn=train_dataset.collate_fn):
    print(features)
    print(labels)
    break

2642
{'dense_features': tensor([[-0.3660],
        [ 0.0000]]), 'reviewerID': tensor([[ 69895],
        [150251]], dtype=torch.int32), 'asin': tensor([[19411],
        [ 3263]], dtype=torch.int32), 'brand': tensor([[3069],
        [1631]], dtype=torch.int32), 'categories': tensor([[356],
        [ 98]], dtype=torch.int32), 'his_asin_seq': tensor([[45384,  -100,  -100],
        [37069, 24828,  9744]], dtype=torch.int32)}
tensor([[1.],
        [1.]])


## Train Model

In [22]:
from torchrec.models import DNN

dnn_hidden_units = [128,64,32]
model = DNN(feat_configs, hidden_units=dnn_hidden_units)
model = model.to(device)
print(model)

==> Model Input: dense_size=1, sparse_size=67
DNN(
  (embeddings): ModuleDict(
    (reviewerID): Embedding(153923, 17)
    (asin): Embedding(62384, 15)
    (brand): Embedding(3504, 11)
    (categories): Embedding(800, 9)
    (his_asin_seq): Embedding(61926, 15)
  )
  (tower): Sequential(
    (0): Linear(in_features=68, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=32, out_features=1, bias=True)
  )
)


In [23]:
optimizer = Adam(model.parameters(),  lr = 0.002, weight_decay = 1e-9)
lr_scd = lr_scheduler.StepLR(optimizer, step_size=len(train_dataloader), gamma=0.8)

In [24]:
from torchrec.trainer import Trainer

trainer = Trainer(
    model, 
    optimizer=optimizer,
    lr_scheduler=lr_scd,
    max_epochs=5,
    early_stopping_rounds=3,
    save_ckpt_path='./ckpt/'
)

model = trainer.fit(train_dataloader, eval_dataloader = test_dataloader, ret_model = 'final') #, init_ckpt_path='./ckpt/')

INFO:Trainer:[Validation] Epoch: 0/5, Validation Loss: {'loss': 0.7358333799976708}
INFO:Trainer:Learning rate: 0.002
INFO:Trainer:[Training] Epoch: 1/5 iter 0/2642, Training Loss: {'loss': 0.7672242522239685}
INFO:Trainer:[Training] Epoch: 1/5 iter 100/2642, Training Loss: {'loss': 0.4826859492063522}
INFO:Trainer:[Training] Epoch: 1/5 iter 200/2642, Training Loss: {'loss': 0.4303086479008198}
INFO:Trainer:[Training] Epoch: 1/5 iter 300/2642, Training Loss: {'loss': 0.4131175540884336}
INFO:Trainer:[Training] Epoch: 1/5 iter 400/2642, Training Loss: {'loss': 0.40101835161447524}
INFO:Trainer:[Training] Epoch: 1/5 iter 500/2642, Training Loss: {'loss': 0.3949488976597786}
INFO:Trainer:[Training] Epoch: 1/5 iter 600/2642, Training Loss: {'loss': 0.39040266101559}
INFO:Trainer:[Training] Epoch: 1/5 iter 700/2642, Training Loss: {'loss': 0.38753292139087403}
INFO:Trainer:[Training] Epoch: 1/5 iter 800/2642, Training Loss: {'loss': 0.3848135073855519}
INFO:Trainer:[Training] Epoch: 1/5 ite

In [25]:
ckpt = trainer.load_ckpt('./ckpt')
model.load_state_dict(ckpt['model'].state_dict())

/mnt/wsl/dev/torchrec/examples/../torchrec/trainer.py:278: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_file)
INFO:Trainer:Loaded model state_dict fr

<All keys matched successfully>

In [26]:
test_preds = []
test_labels = []
model.eval()

for features, labels in test_dataloader:
    outputs = model(features)
    test_preds.append(outputs[:,0])
    test_labels.append(labels[:,0])
test_preds = torch.concat(test_preds, dim=0).detach().cpu().numpy()
test_labels = torch.concat(test_labels, dim=0).detach().cpu().numpy()

In [27]:
print(test_preds.shape, test_labels.shape)

(336650,) (336650,)


In [28]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(test_labels, test_preds)
print("AUC Score:", auc_score)

AUC Score: 0.6766264784924219


# Generate and Test service

In [51]:
# Run this on terminal under root of project
# !python -m torchrec.serve --name dnn --path examples/ckpt/checkpoint.010568.ckpt --dep_paths examples

import os
ret = os.system('curl http://localhost:8000/dnn/health')

{"status":"ok"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0   4004      0 --:--:-- --:--:-- --:--:--  5000


In [52]:
def test_predict(df, name):
    ''' 
    Test the prediction of the model.
    First launch the server by `python -m torchrec.serve --name {name} --path path/to/model`
    '''
    import requests
    import json

    data = {
        'features': df.to_dict(orient='list')
    }
    print(f"Data: {data}")
    data_json = json.dumps(data)
    response = requests.post(
        f"http://localhost:8000/{name}/predict", 
        data=data_json, 
        headers={"Content-Type": "application/json"}
    )

    if response.status_code == 200:
        prediction = response.json()
        print(f"Prediction: {prediction}")
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(f"Response: {response.text}")

In [53]:
if ret == 0:
    test_predict(df_samples.sample(3), name='dnn')

Data: {'features': {'reviewerID': ['A2OIX8THCUGT1H', 'A2V1J3JT5OOZFO', 'A3I9U7LFVD3OYH'], 'asin': ['B000WL6YY8', 'B0038W0K2K', 'B00GPUNJ02'], 'unixReviewTime': [1386547200, 1348704000, 1391472000], 'overall': [4.0, 5.0, 5.0], 'title': ['Cheetah Mounts Plasma LCD Flat Screen TV Articulating Full Motion Dual Arm Wall Mount Bracket For 32-65&quot; Displays Up To 165LBS Black With 10 Foot 1.4 HDMI Cable Fits Up To 24&quot; Studs', 'MEElectronics Sport-Fi M6 Noise-Isolating In-Ear Headphones with Memory Wire (Black)', "iBenzer - Laptop black Sleeve Bag Cover Case for Macbook Pro 13'' with retina display A1425 &amp; A1502 (Macbook Pro 13 Retina, Black)"], 'price': [80.85, 18.99, 8.99], 'brand': ['Cheetah', 'MEElectronics', nan], 'categories': ['TV Ceiling & Wall Mounts', 'Headphones', 'Sleeves & Slipcases'], 'label': [1, 1, 1], 'his_asin_seq': [['B00CDIK908', 'B0052GVNCQ', 'B003UCGDSS', 'B00005LD4T'], ['B003I4FHNA', 'B001F42MKG', 'B0052D38UO', 'B005S6XUXA', 'B0064S7YQU', 'B002FU6KF2', 'B002E